**Note**: If keras is  not already installed, execute: !pip install keras

In [ ]:
# Load required libraries
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Activation
from keras import regularizers
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
time_window = 640;
eeg = tf.keras.layers.Input(shape=[time_window, 64])
env1 = tf.keras.layers.Input(shape=[time_window, 1])
env2 = tf.keras.layers.Input(shape=[time_window, 1])

#add model layers
filters = 1
kernel_size = 16
eeg_conv = tf.keras.layers.Conv1D(filters, kernel_size=kernel_size)(eeg)
cos1 = tf.keras.layers.Dot(1,normalize= True)([eeg_conv , env1[:,:-(kernel_size-1),:]])
cos2 = tf.keras.layers.Dot(1,normalize= True)([eeg_conv , env2[:,:-(kernel_size-1),:]])

# Classification
cos_similarity = tf.keras.layers.Concatenate()([cos1, cos2])
cos_flat = tf.keras.layers.Flatten()(cos_similarity)
out1 = tf.keras.layers.Dense(1, activation="sigmoid")(
    tf.keras.layers.Flatten()(tf.keras.layers.Concatenate()([cos1, cos2])))

# 1 output per batch
out = tf.keras.layers.Reshape([1], name=output_name)(out1)
model = tf.keras.Model(inputs=[eeg, env1, env2], outputs=[out])

In [ ]:
# To check the model summary:
model.summary()

Before we start training the model, we need to make sure that the data is equally balanced. We have attended and unattended envelopes that we give to the model. If we always put the attended envelope at stream 1 and the unattended at stream 2, the model will quickly figure out that it should just always output stream 1 and hence not learn anything. 

The solution to this is to present each segment of EEG twice, where we swap the envelopes, ( and thus, the labels), from place 

In [ ]:

def batch_equalizer(eeg, env_1, env_2, labels):
    # present each of the eeg segments twice, where the envelopes, and thus the labels 
    # are swapped around. EEG presented in small segments [bs, window_length, 64]
    return (np.concatenate([eeg,eeg], axis=0), np.concatenate([env_1, env_2], axis=0),np.concatenate([ env_2, env_1], axis=0)), np.concatenate([labels, (labels+1)%2], axis=0)


* Now we prepare our data to train the model.

In [ ]:
import h5py
import numpy as np

# To load the mat file in v7.3 format. For all previous formats use scipy.io loadmat (https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html)
def load_large_mat(filepath):
    arrays = {}
    f = h5py.File(filepath)
    for k, v in f.items():
        arrays[k] = np.array(v)
    f.close()
    return arrays

In [ ]:
# Preprocessing
def fn_all(fnarrays1,fnarrays2):
    #------add your prerprocessing steps here
    fnxtr_all = fnarrays1
    fny_tr_all = fnarrays2
    x = np.expand_dims(fnxtr_all,-1)
    
    return x,y